Setup

In [1]:
%matplotlib inline

import dataLoader
import matplotlib.pyplot as plt
import torch
import CNN
import itertools

# hyper param search
hyperp = {
    "n_epochs": 3,
    "batchSize": 200,
    "numOfTrials": 3,
    "training_count": 2,
    "validation_count": 1,
    "imgH": 28,
    "imgW": 28,
    "kernels": [16, 32],
    "kernelSize": 5
}

# defining global variables
image_path = "./Fish_toy/images"

# hyper-parameter search
hyperp_grid_params = {
    "dummy": [0], #  keep this when you only need one model
}

Cuda support

In [2]:
# CUDA support 
if torch.cuda.is_available():
    torch.cuda.set_device(0) # 0
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    print("We are using cuda")

We are using cuda


Iterate

In [3]:
import torchvision
from sklearn.metrics import confusion_matrix
from confusion_matrix_plotter import plot_confusion_matrix2
from helper import getModelName
import numpy as np

# get possible experiments
keys, values = zip(*hyperp_grid_params.items())
product = itertools.product(*values)
product_len = len(list(product))

# Just used to print information about data set
dummy = 10
dataLoader.FishDataset(image_path, dummy, dummy, True)
confusionMatrices = []
datasets = []

for v in itertools.product(*values):
    # create experiment params
    experiment_params = dict(zip(keys, v))
    experiment_params = {**hyperp, **experiment_params}
    numOfTrials = experiment_params["numOfTrials"]
    imgH = experiment_params["imgH"]
    imgW = experiment_params["imgW"]
    n_epochs = experiment_params["n_epochs"]
    batchSize = experiment_params["batchSize"]
    training_count = experiment_params["training_count"]
    validation_count = experiment_params["validation_count"]
    kernels = experiment_params["kernels"]
    kernelSize = experiment_params["kernelSize"]
    
    # load images
    dataset = dataLoader.FishDataset(image_path, imgH, imgW)
    datasets.append(dataset)
    numberOfSpecies = len(dataset.getSpeciesList())
    
    confusionMatricesPerExperiment = []
    
    for i in range(numOfTrials):

        # Initialize the prediction and label lists(tensors)
        predlist=torch.zeros(0)
        lbllist=torch.zeros(0)

        # get Loaders
        train_loader, validation_loader, test_loader = dataLoader.getLoadersFromDataset(dataset, training_count, validation_count, batchSize)

        model = CNN.CNN(numberOfSpecies, imgH, imgW, kernels, kernelSize)
        loss_list, accuracy_list = CNN.trainModel(train_loader, validation_loader, n_epochs, model, int(n_epochs/10))


        with torch.no_grad():
            for batch in test_loader:
                inputs = batch["image"]
                classes = batch["class"]
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                predlist=torch.cat([predlist,preds.float().view(-1)])
                lbllist=torch.cat([lbllist,classes.float().view(-1)])

        # Confusion matrix
        confusionMatricesPerExperiment.append(confusion_matrix(lbllist.cpu().numpy(), predlist.cpu().numpy(), labels = range(numberOfSpecies)))
    
    conf_mat = np.mean(confusionMatricesPerExperiment, axis=0) 
    plot_confusion_matrix2(conf_mat,
                              dataset.getSpeciesList(),
                              title=getModelName(experiment_params))

Number of images =  117
Abudefduf vaigiensis  has  4  images
Pardachirus pavoninus  has  5  images
Wetmorella albofasciata  has  5  images
Zebrasoma scopas  has  5  images
Acanthurus achilles  has  5  images
Anampses caeruleopunctatus  has  5  images
Wetmorella nigropinnata  has  5  images
Variola albimarginata  has  4  images
Upeneus moluccensis  has  5  images
Xiphocheilus typus  has  4  images
Acanthurus coeruleus  has  5  images
Acanthopagrus pacificus  has  3  images
Ablabys taenianotus  has  5  images
Acanthurus chirurgus  has  5  images
Amphiprion clarkii  has  5  images
Xenisthmus polyzonatus  has  5  images
Acanthemblemaria spinosa  has  4  images
Acanthemblemaria aspera  has  5  images
Acanthurus bahianus  has  4  images
Anampses geographicus  has  5  images
Amphiprion polymnus  has  5  images
abudefduf septemfasciatus  has  4  images
Lepidozygus tapeinosoma  has  5  images
Xanthichthys auromarginatus  has  5  images
Zanclus cornutus  has  5  images


RuntimeError: CUDA error: out of memory